# Executando código SQL em um dataframe do Pandas

In [8]:
import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv('/content/drive/MyDrive/projeto-ia-codigos/projeto-ia-datasets/preco_casas.csv', sep=';')

In [ ]:
df.head()

,zoneamento,tam_terreno,forma_terreno,qualidade_geral,condicao,ano_construcao,qualidade_aquecedor,ar_condicionado,tam_primeiro_andar,tam_segundo_andar,tam_sala_estar,qtde_quartos,qualidade_cozinha,qtde_banheiros,qtde_comodos,qtde_lareiras,qtde_carros_garagem,tam_garagem,tam_piscina,qualidade_piscina,mes_venda,ano_venda,preco
0,RL,785.03035,Reg,7,5,2003,Ex,Y,79.524968,79.339162,158.864130,3,Gd,30,8,0,2,50.910844,0.0,NaN,2,2008,846510.0
1,RL,891.86880,Reg,6,8,1976,Ex,Y,117.243586,0.000000,117.243586,3,TA,2,6,1,2,42.735380,0.0,NaN,5,2007,NaN
2,RL,1045.15875,IR1,7,5,2001,Ex,Y,85.470760,80.453998,165.924758,3,Gd,2,6,1,2,56.485024,0.0,NaN,9,2008,907410.0
3,RL,887.22365,IR1,7,5,1915,Gd,Y,89.279783,70.234668,159.514451,3,Gd,1,7,1,3,59.643726,0.0,NaN,2,2006,568400.0
4,RL,1324.79678,IR1,8,5,2000,Ex,Y,106.373935,97.826859,204.200794,4,Gd,2,9,1,3,77.666908,0.0,NaN,12,2008,1015000.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Utilizando a biblioteca `pandasql`

[pandasql](https://pypi.org/project/pandasql/) te permite rodar queries SQL nos dataframes do pandas. Ele foi inspirado na biblioteca [sqldf](https://www.rdocumentation.org/packages/sqldf/versions/0.4-11) do R e utiliza a mesma sintaxe do sql encontrado no banco de dados [SQLite](https://www.sqlite.org/index.html).

In [3]:
# primeiro vamos instalar o pandasql: comando copiado da documentação
!pip install -U pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-cp37-none-any.whl size=26820 sha256=f8b42ccb0cd83fd9c0e033a865c0a9b8a2fe7157939f39a34617aa3de519c02c
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


O `pandasql` possui uma função principal chamada `sqldf`. Essa função aceita 2 parâmetros: uma query sql em formato de string e um conjunto de variáveis da sessão/ambiente que está sendo executado no momento. Isso pode ser feito consultando o método `globals()`.

In [4]:
# globals() é um método do python que retorna todas as variáveis globais, códigos e configurações do ambiente atual
globals()

{'In': ['',
  "df = pd.read_csv('/content/drive/MyDrive/Labdata/Aulas - Materiais/projeto-ia-datasets/preco_casas.csv', sep=';')",
  "from google.colab import drive\ndrive.mount('/content/drive')",
  "# primeiro vamos instalar o pandasql: comando copiado da documentação\nget_ipython().system('pip install -U pandasql')",
  '# globals() é um método do python que retorna todas as variáveis globais, códigos e configurações do ambiente atual\nglobals()'],
 'Out': {},
 '_': '',
 '__': '',
 '___': '',
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__loader__': None,
 '__name__': '__main__',
 '__package__': None,
 '__spec__': None,
 '_dh': ['/content'],
 '_exit_code': 0,
 '_i': '# primeiro vamos instalar o pandasql: comando copiado da documentação\n!pip install -U pandasql',
 '_i1': "df = pd.read_csv('/content/drive/MyDrive/Labdata/Aulas - Materiais/projeto-ia-dat

In [5]:
from pandasql import sqldf

# definindo uma função utilizando a notação `lambda`
pysqldf = lambda q: sqldf(q, globals())

# definindo uma função do jeito padrão
# def pysqldf(q):
#    return sqldf(q, globals())

In [10]:
# com a função pysqldf() definida, podemos chamá-la passando como argumento a query que queremos executar
pysqldf("SELECT * FROM df LIMIT 5;")

,zoneamento,tam_terreno,forma_terreno,qualidade_geral,condicao,ano_construcao,qualidade_aquecedor,ar_condicionado,tam_primeiro_andar,tam_segundo_andar,tam_sala_estar,qtde_quartos,qualidade_cozinha,qtde_banheiros,qtde_comodos,qtde_lareiras,qtde_carros_garagem,tam_garagem,tam_piscina,qualidade_piscina,mes_venda,ano_venda,preco
0,RL,785.03035,Reg,7,5,2003,Ex,Y,79.524968,79.339162,158.864130,3,Gd,30,8,0,2,50.910844,0.0,None,2,2008,846510.0
1,RL,891.86880,Reg,6,8,1976,Ex,Y,117.243586,0.000000,117.243586,3,TA,2,6,1,2,42.735380,0.0,None,5,2007,NaN
2,RL,1045.15875,IR1,7,5,2001,Ex,Y,85.470760,80.453998,165.924758,3,Gd,2,6,1,2,56.485024,0.0,None,9,2008,907410.0
3,RL,887.22365,IR1,7,5,1915,Gd,Y,89.279783,70.234668,159.514451,3,Gd,1,7,1,3,59.643726,0.0,None,2,2006,568400.0
4,RL,1324.79678,IR1,8,5,2000,Ex,Y,106.373935,97.826859,204.200794,4,Gd,2,9,1,3,77.666908,0.0,None,12,2008,1015000.0


In [11]:
query = """
SELECT ar_condicionado,
        count(ar_condicionado) as quantidade
FROM df
GROUP BY ar_condicionado
"""

pysqldf(query)

,ar_condicionado,quantidade
0,N,95
1,Y,1365


In [12]:
(
    df
    .groupby('ar_condicionado')
    .agg(quantidade = ('ar_condicionado','count'))
    .reset_index()
)

,ar_condicionado,quantidade
0,N,95
1,Y,1365


## Criando um banco de dados

Podemos criar um banco de dados para salvar o nosso dataframe com o objetivo de executarmos queries nesse banco de dados diretamente nesse banco de dados. Para isso, podemos utilizar um pacote chamado [sqlalchemy](https://www.sqlalchemy.org/), que permite que o possamos criar e gerenciar bancos de dados utilizando python.

In [13]:
# importando o pacote sqlalchemy para criação e manipulação de bancos de dados utilizando python
from sqlalchemy import create_engine

In [14]:
# criando um banco de dados chamado preco_casas.db
engine = create_engine('sqlite:///preco_casas.db', echo=True)

In [15]:
# conectando ao banco de dados
sqlite_connection = engine.connect()

2021-03-19 00:23:26,262 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-19 00:23:26,265 INFO sqlalchemy.engine.base.Engine ()
2021-03-19 00:23:26,273 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-19 00:23:26,275 INFO sqlalchemy.engine.base.Engine ()


In [16]:
# salvando o dataframe df como uma tabela do banco de dados
nome_da_tabela = "tb_preco_casas"
df.to_sql(nome_da_tabela, sqlite_connection, if_exists='fail')

2021-03-19 00:24:11,926 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("tb_preco_casas")
2021-03-19 00:24:11,928 INFO sqlalchemy.engine.base.Engine ()
2021-03-19 00:24:11,931 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("tb_preco_casas")
2021-03-19 00:24:11,933 INFO sqlalchemy.engine.base.Engine ()
2021-03-19 00:24:11,946 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE tb_preco_casas (
	"index" BIGINT, 
	zoneamento TEXT, 
	tam_terreno FLOAT, 
	forma_terreno TEXT, 
	qualidade_geral BIGINT, 
	condicao BIGINT, 
	ano_construcao BIGINT, 
	qualidade_aquecedor TEXT, 
	ar_condicionado TEXT, 
	tam_primeiro_andar FLOAT, 
	tam_segundo_andar FLOAT, 
	tam_sala_estar FLOAT, 
	qtde_quartos BIGINT, 
	qualidade_cozinha TEXT, 
	qtde_banheiros BIGINT, 
	qtde_comodos BIGINT, 
	qtde_lareiras BIGINT, 
	qtde_carros_garagem BIGINT, 
	tam_garagem FLOAT, 
	tam_piscina FLOAT, 
	qualidade_piscina TEXT, 
	mes_venda BIGINT, 
	ano_venda BIGINT, 
	preco FLOAT
)


2021-03-19 00:24:11,947 INFO 

In [17]:
# fechando a conexão
sqlite_connection.close()

In [18]:
# uma vez com o banco de dados criado, podemos conectar novamente e executarmos queries sql
# como o banco de dados já existe, o comando create_engine apenas lê o banco de dados existente
engine = create_engine('sqlite:///preco_casas.db')
sqlite_connection = engine.connect()

In [19]:
# executando uma query com o método pd.read_sql
df = pd.read_sql("select * from tb_preco_casas limit 5", con=sqlite_connection)
df.head()

,index,zoneamento,tam_terreno,forma_terreno,qualidade_geral,condicao,ano_construcao,qualidade_aquecedor,ar_condicionado,tam_primeiro_andar,tam_segundo_andar,tam_sala_estar,qtde_quartos,qualidade_cozinha,qtde_banheiros,qtde_comodos,qtde_lareiras,qtde_carros_garagem,tam_garagem,tam_piscina,qualidade_piscina,mes_venda,ano_venda,preco
0,0,RL,785.03035,Reg,7,5,2003,Ex,Y,79.524968,79.339162,158.864130,3,Gd,30,8,0,2,50.910844,0.0,None,2,2008,846510.0
1,1,RL,891.86880,Reg,6,8,1976,Ex,Y,117.243586,0.000000,117.243586,3,TA,2,6,1,2,42.735380,0.0,None,5,2007,NaN
2,2,RL,1045.15875,IR1,7,5,2001,Ex,Y,85.470760,80.453998,165.924758,3,Gd,2,6,1,2,56.485024,0.0,None,9,2008,907410.0
3,3,RL,887.22365,IR1,7,5,1915,Gd,Y,89.279783,70.234668,159.514451,3,Gd,1,7,1,3,59.643726,0.0,None,2,2006,568400.0
4,4,RL,1324.79678,IR1,8,5,2000,Ex,Y,106.373935,97.826859,204.200794,4,Gd,2,9,1,3,77.666908,0.0,None,12,2008,1015000.0


In [ ]:
# executando uma query com o método pd.read_sql_query
df = pd.read_sql_query("select * from tb_preco_casas limit 5", con=sqlite_connection)
df.head()

,index,zoneamento,tam_terreno,forma_terreno,qualidade_geral,condicao,ano_construcao,qualidade_aquecedor,ar_condicionado,tam_primeiro_andar,tam_segundo_andar,tam_sala_estar,qtde_quartos,qualidade_cozinha,qtde_banheiros,qtde_comodos,qtde_lareiras,qtde_carros_garagem,tam_garagem,tam_piscina,qualidade_piscina,mes_venda,ano_venda,preco
0,0,RL,785.03035,Reg,7,5,2003,Ex,Y,79.524968,79.339162,158.864130,3,Gd,30,8,0,2,50.910844,0.0,None,2,2008,846510.0
1,1,RL,891.86880,Reg,6,8,1976,Ex,Y,117.243586,0.000000,117.243586,3,TA,2,6,1,2,42.735380,0.0,None,5,2007,NaN
2,2,RL,1045.15875,IR1,7,5,2001,Ex,Y,85.470760,80.453998,165.924758,3,Gd,2,6,1,2,56.485024,0.0,None,9,2008,907410.0
3,3,RL,887.22365,IR1,7,5,1915,Gd,Y,89.279783,70.234668,159.514451,3,Gd,1,7,1,3,59.643726,0.0,None,2,2006,568400.0
4,4,RL,1324.79678,IR1,8,5,2000,Ex,Y,106.373935,97.826859,204.200794,4,Gd,2,9,1,3,77.666908,0.0,None,12,2008,1015000.0


In [ ]:
# executando uma query com o método pd.read_table
df = pd.read_sql_table("tb_preco_casas", con=sqlite_connection)
df.head()

,index,zoneamento,tam_terreno,forma_terreno,qualidade_geral,condicao,ano_construcao,qualidade_aquecedor,ar_condicionado,tam_primeiro_andar,tam_segundo_andar,tam_sala_estar,qtde_quartos,qualidade_cozinha,qtde_banheiros,qtde_comodos,qtde_lareiras,qtde_carros_garagem,tam_garagem,tam_piscina,qualidade_piscina,mes_venda,ano_venda,preco
0,0,RL,785.03035,Reg,7,5,2003,Ex,Y,79.524968,79.339162,158.864130,3,Gd,30,8,0,2,50.910844,0.0,None,2,2008,846510.0
1,1,RL,891.86880,Reg,6,8,1976,Ex,Y,117.243586,0.000000,117.243586,3,TA,2,6,1,2,42.735380,0.0,None,5,2007,NaN
2,2,RL,1045.15875,IR1,7,5,2001,Ex,Y,85.470760,80.453998,165.924758,3,Gd,2,6,1,2,56.485024,0.0,None,9,2008,907410.0
3,3,RL,887.22365,IR1,7,5,1915,Gd,Y,89.279783,70.234668,159.514451,3,Gd,1,7,1,3,59.643726,0.0,None,2,2006,568400.0
4,4,RL,1324.79678,IR1,8,5,2000,Ex,Y,106.373935,97.826859,204.200794,4,Gd,2,9,1,3,77.666908,0.0,None,12,2008,1015000.0


Por fim, podemos baixar o banco de dados criado `preco_casas.db` que contém a nossa tabela que era um dataframe do pandas, e abrir esse banco de dados em qualquer programa que nos permita abrir e executar queries em bancos de dados relacionais. Como exemplo para bancos do tipo `sqlite`, temos o [DB Browser](https://sqlitebrowser.org/).

# Web Scrapping simples com Pandas

Podemos baixar diretamente para um dataframe do pandas qualquer tabela que esteja disponível em uma página HTML (esse método nem sempre irá funcionar, mas como é um método rápido, vale a pena tentar!)

In [20]:
# o comando pd.read_html irá retornar uma lista com todas as tabelas que ele conseguir encontrar na página
lista_de_tabelas = pd.read_html('https://pt.wikipedia.org/wiki/Lista_de_estados_brasileiros_por_n%C3%BAmero_de_munic%C3%ADpios')

In [21]:
# retornou 5 tabelas
len(lista_de_tabelas)

5

In [22]:
# primeira tabela
lista_de_tabelas[0]

,Número total de municípios,Número total de municípios.1
0,Brasil,5 568


In [23]:
# segunda tabela
lista_de_tabelas[1]

,Região,Unidadesfederativas,Municípios,Porcentagem
0,Nordeste,9,1 793,"32,21%"
1,Sudeste,4,1 668,"29,95%"
2,Sul,3,1 191,"21,39%"
3,Centro-Oeste,4,466,"8,37%"
4,Norte,7,450,"8,08%"


In [24]:
# terceira tabela
lista_de_tabelas[2]

,Posição,Estado,Região,Número demunicípios[8],Número de habitantespor estado federado[9],Média de habitantespor município
0,1,Minas Gerais,Sudeste,853,20 989 259,"24 606,40"
1,2,São Paulo,Sudeste,645,44 744 199,"69 370,85"
2,3,Rio Grande do Sul,Sul,497,11 290 773,"22 717,85"
3,4,Bahia,Nordeste,417,15 271 073,"36 621,28"
4,5,Paraná,Sul,399,11 241 665,"28 174,60"
5,6,Santa Catarina,Sul,295,6 882 793,"23 331,50"
6,7,Goiás,Centro-Oeste,246,6 690 173,"27 195,83"
7,8,Piauí,Nordeste,224,3 212 374,"14 340,96"
8,9,Paraíba,Nordeste,223,3 995 541,"17 917,22"
9,10,Maranhão,Nordeste,217,6 945 547,"32 007,13"


In [25]:
# quarta tabela
lista_de_tabelas[3]

,vdeClassificações das unidades federativas do Brasil,vdeClassificações das unidades federativas do Brasil.1
0,Características geográficas,Área Densidade demográfica Número de município...
1,Indicadores demográficos,População Densidade demográfica Natalidade Fec...
2,Gestão pública,Sedes governamentais Transparência Arrecadação...
3,Indicadores econômicos e sociais,PIB Série histórica 1939 1950 1960 1970 1980 1...
4,PIB,Série histórica 1939 1950 1960 1970 1980 1990 ...
5,Trabalho,Empregados com carteira assinada Desemprego Ap...
6,Desigualdade e bem-estar,Segurança alimentar Renda per capita IDH segun...
7,Educação,Alfabetização Crianças e adolescentes fora da ...
8,Saúde,Mortalidade infantil Óbitos infantis Taxa de h...
9,Outros,Religião Circunscrições eclesiásticas Competit...


In [26]:
# quinta tabela
lista_de_tabelas[4]

,0,1
0,PIB,Série histórica 1939 1950 1960 1970 1980 1990 ...
1,Trabalho,Empregados com carteira assinada Desemprego Ap...
2,Desigualdade e bem-estar,Segurança alimentar Renda per capita IDH segun...
3,Educação,Alfabetização Crianças e adolescentes fora da ...
4,Saúde,Mortalidade infantil Óbitos infantis Taxa de h...
5,Outros,Religião Circunscrições eclesiásticas Competit...
